# Estrazione di entità dal testo e linked data

Carico il file di testo

In [1]:
import requests
import re
import json
import pandas as pd
list_of_lines = []
with open("data/t1.txt", "r") as f:
    list_of_lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'data/t1.txt'

Seleziono solo l'ultima frase

In [118]:
last_sentence = list_of_lines[-1:]
list_of_lines_clean = [ el.strip() for el in list_of_lines if len(el.replace("\n", "")) > 4] 
df_lines = pd.DataFrame({'text' : list_of_lines_clean })

In [119]:
df_lines

,text
0,ï»¿We look to fiction for eternal truths about...
1,More like this:
2,- Inside the mind of a literary hoaxer
3,- Primo Levi: A clear-eyed view of evil ...
4,- The sci-fi taking on climate change
5,"In his 1968 novel Stand on Zanzibar, for insta..."
6,Born in 1934 in the Thames riverside hamlet of...


In [71]:
print(last_sentence)

['Born in 1934 in the Thames riverside hamlet of Preston Crowmarsh, Oxfordshire, John Kilian Houston Brunner was just six years old when he discovered science fiction. As Professor Jad Smith relates in his comprehensive study, John Brunner, with World War Two raging, the family moved to Herefordshire, where Brunner\'s father intended to support the war effort by running a farm. In the chaos of the move, his grandfather\'s rare 1898 edition of HG Wells\' The War of the Worlds ended up shelved in the playroom. Brunner devoured it and from that moment, as he would later explain in a short autobiography, was imprinted by the genre "as permanently as one of Konrad Lorenz\'s geese".']


definisco l'endpoint e i dati da passare

In [161]:
from urllib import parse
import json
import requests

def enrich_text(last_sentence):
    
    url = "http://api.dbpedia-spotlight.org/en/annotate"
    my_dict = { 
        "text" : last_sentence,
        "confidence" : 0.5 
    }
    response = requests.post("https://api.dbpedia-spotlight.org/en/annotate", data=my_dict, headers={"Accept":"application/json"})
    
    

    if (response.status_code == 200):
        content = json.loads(response.content)
    else:
        return None
    
    if "Resources" in content.keys():
        content = content["Resources"]
    else:
        content = []
    return content

In [162]:

contenuto = enrich_text("Calais")
#df_lines["Entities"] = df_lines["text"].apply(lambda x : [ el["@surfaceForm"] for el in enrich_text(x) if el is not None ])


In [163]:
print(contenuto)

[{'@URI': 'http://dbpedia.org/resource/Calais', '@support': '3650', '@types': 'Wikidata:Q486972,Schema:Place,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Settlement', '@surfaceForm': 'Calais', '@offset': '0', '@similarityScore': '0.9990351222256937', '@percentageOfSecondRank': '4.3632776711395816E-4'}]


In [192]:
response = requests.post('http://dbpedia.org/resource/Italy',headers={"Accept":"application/json"})
content = json.loads(response.content)
#print(content)
for j in content["http://dbpedia.org/resource/France"]:
    if(j == "http://dbpedia.org/ontology/abstract"):
        for cont in content["http://dbpedia.org/resource/Italy"][j]:
            if cont["lang"]=="it":
                print(cont['value'])

L'Italia (/iˈtalja/, ), ufficialmente Repubblica Italiana, è una repubblica parlamentare situata nell'Europa meridionale, con una popolazione di circa 60,4 milioni di abitanti e Roma come capitale. La parte continentale, delimitata dall'arco alpino, confina a nord, da ovest a est, con Francia, Svizzera, Austria e Slovenia; il resto del territorio, circondato dai mari Ligure, Tirreno, Ionio e Adriatico, si protende nel mar Mediterraneo, occupando la penisola italiana e numerose isole (le maggiori sono Sicilia e Sardegna), per un totale di 302 072,84 km². Gli Stati della Città del Vaticano e di San Marino sono enclavi della Repubblica mentre Campione d'Italia è l'unica exclave italiana. Roma, che fu capitale dell'Impero romano, è stata per secoli il centro politico e culturale della civiltà occidentale. Dopo la caduta dell'Impero romano d'Occidente, l'Italia medievale fu soggetta a invasioni e dominazioni di popolazioni germaniche, come gli Ostrogoti, i Longobardi e i Normanni. Nel XV se

In [165]:
import json
#from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

## initial consts
BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
TEXT = "Tolerance, safety and efficacy of Hedera helix extract in inflammatory bronchial diseases under clinical practice conditions: a prospective, open, multicentre postmarketing study in 9657 patients.     In this postmarketing study 9657 patients (5181 children) with bronchitis (acute or chronic bronchial inflammatory disease) were treated with a syrup containing dried ivy leaf extract. After 7 days of therapy, 95% of the patients showed improvement or healing of their symptoms. The safety of the therapy was very good with an overall incidence of adverse events of 2.1% (mainly gastrointestinal disorders with 1.5%). In those patients who got concomitant medication as well, it could be shown that the additional application of antibiotics had no benefit respective to efficacy but did increase the relative risk for the occurrence of side effects by 26%. In conclusion, it is to say that the dried ivy leaf extract is effective and well tolerated in patients with bronchitis. In view of the large population considered, future analyses should approach specific issues concerning therapy by age group, concomitant therapy and baseline conditions."
CONFIDENCE = '0.5'
SUPPORT = '10'
REQUEST = BASE_URL.format(
    text=urllib.parse.quote_plus(TEXT), 
    confidence=CONFIDENCE, 
    support=SUPPORT
)
HEADERS = {'Accept': 'application/json'}
#sparql = SPARQLWrapper("http://dbpedia.org/sparql")
all_urls = []

r = requests.get(url=REQUEST, headers=HEADERS)
response = r.json()
resources = response['Resources']
for res in resources:
    all_urls.append(res['@URI'])
print(all_urls)

['http://dbpedia.org/resource/Hedera', 'http://dbpedia.org/resource/Bronchitis', 'http://dbpedia.org/resource/Cold_medicine', 'http://dbpedia.org/resource/Hedera', 'http://dbpedia.org/resource/After_7', 'http://dbpedia.org/resource/Notifiable_disease', 'http://dbpedia.org/resource/Gastrointestinal_disease', 'http://dbpedia.org/resource/Antibiotic', 'http://dbpedia.org/resource/Risk_ratio', 'http://dbpedia.org/resource/Hedera', 'http://dbpedia.org/resource/Bronchitis']


In [3]:
import json
import socket
s=socket.socket()  #creo il socket
s.connect(("www.dbpedia.org",80)) #connessione a google
#response = requests.post('http://dbpedia.org/resource/Italy',headers={"Accept":"application/json"})
richiesta = 'POST https://dbpedia.org/resource/Italy'
print("ciao1")
s.send(richiesta.encode())
print("ciao2")
response_TCP =  s.recv(2048)
print(response_TCP)
content = json.loads(response_TCP.content)
#print(content)
for j in content["http://dbpedia.org/resource/Italy"]:
    if(j == "http://dbpedia.org/ontology/abstract"):
        for cont in content["http://dbpedia.org/resource/Italy"][j]:
            if cont["lang"]=="it":
                print(cont['value'])

ciao1
ciao2
b''


AttributeError: 'bytes' object has no attribute 'content'